In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import models

In [ ]:
train_path = "../input/plant-pathology-2021-fgvc8/train_images/"

In [ ]:
train_csv = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
train_csv.tail(7)

In [ ]:
print(len(train_csv))

print(len(os.listdir("../input/plant-pathology-2021-fgvc8/train_images")))
print(len(os.listdir("../input/plant-pathology-2021-fgvc8/test_images")))

In [ ]:
train_csv.labels.nunique()

In [ ]:
train_csv.labels.value_counts()

In [ ]:
f, axarr = plt.subplots(2,2)
idx = [np.random.randint(18632) for _ in range(4)]
axarr[0,0].imshow(plt.imread(train_path + train_csv.iloc[idx[0],0]))
axarr[0,1].imshow(plt.imread(train_path + train_csv.iloc[idx[1],0]))
axarr[1,0].imshow(plt.imread(train_path + train_csv.iloc[idx[2],0]))
axarr[1,1].imshow(plt.imread(train_path + train_csv.iloc[idx[3],0]))

print(train_csv.iloc[idx[0],1],",", train_csv.iloc[idx[1],1],",", train_csv.iloc[idx[2],1],",", train_csv.iloc[idx[3],1])

In [ ]:
for i in range(50):
    print(plt.imread(train_path + train_csv.iloc[i,0]).shape)

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
        #tf.keras.layers.experimental.preprocessing.Resizing(3000, 4000),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.3),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    ]
)

# Add the image to a batch
image = tf.expand_dims(plt.imread(train_path + train_csv.iloc[idx[0],0]), 0)

plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")


In [ ]:
#!cp -r "../input/plant-pathology-2021-fgvc8/train_images/" "./train_images/"

In [ ]:
!mkdir train_images

In [ ]:
from tqdm import tqdm

sample = pd.DataFrame(train_csv.labels.value_counts())#.loc["scab"]
cols = {i for i in train_csv.labels if sample.loc[i][0] < 700}

aug = dict()

for col in cols:
    aug[col] = []
    
    for i in tqdm(range(18632), total=18632):
        if train_csv.iloc[i, 1] == col:
            aug[col].append(train_csv.iloc[i, 0])
            

In [ ]:
def combine(val):
    return val[0]+"_"+val[1]
    
for i in aug:
    print(combine(i.split(" ")))

In [ ]:
augmentations = tf.keras.Sequential([
    
    tf.keras.layers.experimental.preprocessing.RandomContrast(0.3),
    tf.keras.layers.experimental.preprocessing.Resizing(3000, 4000),
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.5, fill_mode='reflect', interpolation='bilinear'),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.4),
    #tf.keras.layers.experimental.preprocessing.Rescaling(1./255, offset=0.0),#[0,1]
    #tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1)#[-1,1]
    
])
out_path = "./train_images/"

for i in aug:
    rem = 1000 - len(aug[i])
    for j in tqdm(range(rem), total=rem):
        image = np.random.choice(aug[i])
        image = tf.expand_dims(plt.imread(train_path + image), 0)
        augmented_image = augmentations(image)
        img_name = combine(i.split(" "))
        img_path = out_path + f"{img_name}_{j}.jpg"
        train_csv = train_csv.append(pd.DataFrame({"image":f"{img_name}_{j}.jpg", "labels":i},index=[0]), ignore_index=True)
        tf.keras.preprocessing.image.save_img(img_path, np.squeeze(augmented_image), data_format="channels_last")

In [ ]:
train_csv.to_csv("train_aug.csv", index=False)

In [ ]:
len(os.listdir("./train_images"))

In [ ]:
!zip -r aug_images.zip "./"